# Pair Programming ETL III

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [14]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import requests
import pandas as pd
import numpy as np
import ast

In [ ]:
dicc = {"usa": [39.7837304, -100.445882], "australia":[-24.7761086, 134.755], "south africa":[-28.8166236, 24.991639],"new zealand": [-41.5000831, 172.8344077],"papua new guinea": [-5.6816069, 144.2489081] }

In [19]:
class extraer_limpiar:
    
    def __init__(self, lat, lon, pais):
        
        self.lat = lat
        self.lon = lon
        self.pais = pais         

    def extraer_datos(self):
        
        df = pd.DataFrame()
    
        url ="http://www.7timer.info/bin/api.pl?lon={self.lon}&lat={self.lat}&product=meteo&output=json"
    
        response = requests.get(url = url) 
        response.status_code
    
        response_df = pd.DataFrame.from_dict(pd.json_normalize(response.json()["dataseries"]))
        response_df["country"] = self.pais
    
        df = pd.concat([df, response_df], axis = 0)
        return df 
    
    def limpiar_rh(self,df):
        
        df['rh_profile']= df['rh_profile'].apply(ast.literal_eval)

        x = df['rh_profile'].apply(pd.Series) 

        for i in range(len(x.columns)): 
            nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 
            valores = list(x[i].apply(pd.Series)["rh"] )
     
            df.insert(i, nombre, valores)
        return df
    
    def limpiar_wind(self,df2):
    
        df2['wind_profile']= df2['wind_profile'].apply(ast.literal_eval)

        y = df2['wind_profile'].apply(pd.Series)

        for i in range(len(y.columns)): 
            nombre = "winddirection_" + str(y[i].apply(pd.Series)["layer"][0])
            valores = list(y[i].apply(pd.Series)["direction"])

            df2.insert(i, nombre, valores)
    
        return df2

In [20]:
api = extraer_limpiar(39.7837304, -100.445882,"usa")
api

In [23]:
api2 = api.extraer_datos()
api2.head(1)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 180, 'speed':...",27,-1,10,-9999,rain,2,0,195,2,usa


In [24]:
rh = api.limpiar_wind(api2)


ValueError: malformed node or string: [{'layer': '950mb', 'direction': 180, 'speed': 2}, {'layer': '900mb', 'direction': 150, 'speed': 2}, {'layer': '850mb', 'direction': 70, 'speed': 2}, {'layer': '800mb', 'direction': 85, 'speed': 2}, {'layer': '750mb', 'direction': 95, 'speed': 2}, {'layer': '700mb', 'direction': 95, 'speed': 3}, {'layer': '650mb', 'direction': 95, 'speed': 3}, {'layer': '600mb', 'direction': 85, 'speed': 3}, {'layer': '550mb', 'direction': 70, 'speed': 3}, {'layer': '500mb', 'direction': 50, 'speed': 4}, {'layer': '450mb', 'direction': 80, 'speed': 5}, {'layer': '400mb', 'direction': 95, 'speed': 3}, {'layer': '350mb', 'direction': 90, 'speed': 4}, {'layer': '300mb', 'direction': 115, 'speed': 5}, {'layer': '250mb', 'direction': 115, 'speed': 5}, {'layer': '200mb', 'direction': 125, 'speed': 4}]